In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 6.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/libindic/syllabalizer.git

Cloning into 'syllabalizer'...
remote: Enumerating objects: 186, done.
remote: Total 186 (delta 0), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (186/186), 41.35 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [ ]:
%cd syllabalizer
%run setup.py sdist

/content/syllabalizer


/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separ

In [ ]:
!pip install dist/libindic-syllabifier*.tar.gz
!pip install libindic-utils

Processing ./dist/libindic-syllabifier-0.0.1.dev28.tar.gz
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-syllabifier: filename=libindic_syllabifier-0.0.1.dev28-py2.py3-none-any.whl size=8122 sha256=26d44e325dd61bdfe9c362642138b17db181cb03d99008712ebc48f5d0cd9929
  Stored in directory: /root/.cache/pip/wheels/d2/50/19/e2620530fc718493df9f5bca90d23db025cc864b9c7c0a98ef
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8465 sha256=3ae8e58a9f630fc6e94114aa187e3beb5c820a1182ce413b66889cea23f712c2
  Stored in directory: /root/.cache/pip/wheels/c0/72/43/0c779b79d708c78240beb3b0bb8f5ff3c2ab81c4e5271ea1aa
Successfully built libindic-syllabifier silpa_common
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-utils: filename=libindic_utils-1.0.3-py3-none-any.whl size=9685 sha256=98293ef414ff0523016d3c6a4bd1153d6d3eac4c04a9bbbdbff2222156067d04
  Stored in directory: /root/.cache/pip/w

In [ ]:
from libindic.syllabifier import Syllabifier

def syllable(word):
    instance = Syllabifier()
    try:
        result = instance.syllabify(word)
    except:
        result = word
    return result


In [ ]:
#importing all the needed libraries
import pandas as pd
import numpy as np
import nltk
import sklearn

import scipy.stats
import math, string, re

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
#function to extract features
def word2features(sent, i):
    wordc = sent[i][0]
    word = syllable(wordc)

    features = {
        'bias': 1.0,   #1.0
        'word': word,
        'len(word)': len(word),
        'word[:4]': word[:4],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[-4:]': word[-4:],
        'word.ispunctuation': (wordc in string.punctuation),
    }
    if i > 0:
        word1l = sent[i-1][0]
        word1 = syllable(word1l)
        features.update({
            '-1:word': word1,
            '-1:len(word)': len(word1),
            '-1:word[:3]': word1[:3],
            '-1:word[:2]': word1[:2],
            '-1:word[-3:]': word1[-3:],
            '-1:word[-2:]': word1[-2:],
            '-1:word.ispunctuation': (word1l in string.punctuation),
        })
    else:
        features['BOS'] = True

    if i > 1:
        word2l = sent[i-2][0]
        word2 = syllable(word2l)

        features.update({
            '-2:word': word2,
            '-2:len(word)': len(word2),
            '-2:word[:3]': word2[:3],
            '-2:word[:2]': word2[:2],
            '-2:word[-3:]': word2[-3:],
            '-2:word[-2:]': word2[-2:],
            '-2:word.ispunctuation': (word2l in string.punctuation),
        })

    if i < len(sent)-1:
        word1r = sent[i+1][0]
        word1 = syllable(word1r)
        features.update({
            '+1:word': word1,
            '+1:len(word)': len(word1),
            '+1:word[:3]': word1[:3],
            '+1:word[:2]': word1[:2],
            '+1:word[-3:]': word1[-3:],
            '+1:word[-2:]': word1[-2:],
            '+1:word.ispunctuation': (word1r in string.punctuation),
        })
    else:
        features['EOS'] = True

    if i < len(sent) - 2:
        word2r = sent[i+2][0]
        word2 = syllable(word2r)
        features.update({
            '+2:word': word2,
            '+2:len(word)': len(word2),
            '+2:word[:3]': word2[:3],
            '+2:word[:2]': word2[:2],
            '+2:word[-3:]': word2[-3:],
            '+2:word[-2:]': word2[-2:],
            '+2:word.ispunctuation': (word2r in string.punctuation),
        })

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word[1] for word in sent]

In [ ]:
#formatting the data into sentences
def format_data(data):
    sents = []
    for i in range(len(data)):
        if math.isnan(data.iloc[i, 0]):
            continue
        elif data.iloc[i, 0] == 1.0:
            sents.append([[data.iloc[i, 1], data.iloc[i, 2]]])
        else:
          if data.iloc[i, 1] == '*,':    # new line added
            data.iloc[i, 1] = '.'  # to remove * from *, - added to modify . earlier - new line added
          sents[-1].append([data.iloc[i, 1], data.iloc[i, 2]])
    for sent in sents:
        for i, word in enumerate(sent):
            if type(word[0]) != str:
                del sent[i]

    Xtrain = [sent2features(s) for s in sents]
    ytrain = [sent2labels(s) for s in sents]

    return Xtrain, ytrain #sents

In [ ]:
#formatting the data into sentences
def test_format_data(data):
    sents = []
    for i in range(len(data)):
        if math.isnan(data.iloc[i, 0]):
            continue
        elif data.iloc[i, 0] == 1.0:
            sents.append([data.iloc[i, 1]])
        else:
          if data.iloc[i, 1] == '*,':    # new line added
            data.iloc[i, 1] = '.'  # to remove * from *, - added to modify . earlier - new line added
          sents[-1].append([data.iloc[i, 1]])
    for sent in sents:
        for i, word in enumerate(sent):
            if type(word[0]) != str:
                del sent[i]

    Xtest = [sent2features(s) for s in sents]

    return Xtest  #sents

In [ ]:
def txt_format(list):
  words= []
  tags= []

  for line in list:
    words.append(line[0])
    tags.append(line[1])
  zip_data = zip(words, tags)

  line_no = []
  lwords=[]
  ltags=[]
  i=1
  for word, tag in zip_data:   #(words, tags):
    if word != '*':
        line_no.append(float(i))
        lwords.append(word)
        ltags.append(tag)
        i += 1
    else:
        line_no.append(np.NaN)
        lwords.append(np.NaN)
        ltags.append(np.NaN)
        i=1

  dict = {'ID': line_no, 'WORD': lwords, 'TAG': ltags}
  df = pd.DataFrame(dict)
  return df

In [ ]:
def test_txt_format(list):
  words= []
  line_no = []
  lwords=[]
  ltags=[]
  i=1

  for word in list:
    if word != '*':
        line_no.append(float(i))
        lwords.append(word)
        i += 1
    else:
        line_no.append(np.NaN)
        lwords.append(np.NaN)
        i=1
  dict = {'ID': line_no, 'WORD': lwords}
  df = pd.DataFrame(dict)
  return df

In [ ]:
# read the tsv file line by line and replace the blank lines by '*' and its tag also as *

def process_blanklines(path):
  rp_text = ['*', '*\n']   # to replace blank lines by * and its tag also as *
  line_list = []

  with open(path) as f:
  # Read data line by line
    for line in f:
      l = line.split('\t')   # split data by tab and store it in list
      if len(l) == 1:    # blank lines - only \n is present
        l = rp_text
      temp = l[1].split('\n')
      l[1] = temp[0]   #l[1].replace("\n","")

      line_list.append(l)

  # to modify nosiy data - process . in between the text - replacing . by , to kep it simple
  for line in range(len(line_list)-1):
    if line_list[line][0] == '.' and line_list[line+1][0] != '*':
      line_list[line][0] = '*,'

  return txt_format(line_list)

In [ ]:
# read the tsv file line by line and replace the blank lines by '*' and its tag also as *

def test_process_blanklines(path):
  rp_text = ["*\n"]   # to replace blank lines by * and its tag also as *

  line_list = []

  # open .tsv file
  with open(path) as f:

  # Read data line by line
    for line in f:
      l = line.split('\n')   # split data by tab and store it in list
      if l[0] == '""':       #'\n':    # blank lines - only \n is present
        l[0] = '*'           #rp_text
      line_list.append(l[0])

  # to modify nosiy data - process . in between the text - replacing . by , to kep it simple
  for line in range(len(line_list)-1):
    if line_list[line] == '.' and line_list[line+1] !='*':
      line_list[line] = '*,'

  return test_txt_format(line_list)

In [ ]:
# flatten the list of lists of pedictions
def flatten(tags):
  p_tags = []
  for lines in tags:
    for tag in lines:
      p_tags.append(tag)
    p_tags.append('')
  p_tags = p_tags[:-1]
  return p_tags

In [ ]:
#%%time
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm = 'lbfgs',
    c1 = 0.1,  #065,
    c2 = 0.1,  #  002,
    max_iterations = 200,   #200,
    all_possible_transitions=True
)


Process the datafiles

In [ ]:
# to process part 1 and 2 train sets
train1 = process_blanklines("/content/drive/MyDrive/disfluency/kannada_train.tsv")
train2 = process_blanklines("/content/drive/MyDrive/disfluency/kannada_train_2.tsv")

In [ ]:
train = pd.concat([train1, train2], ignore_index=True)   # to concatenate two training files
train = train[1:]

In [ ]:
train.head(20)

,ID,WORD,TAG
1,1.0,ಹೌದು,B-false_R
2,2.0,ಅದೇ,I-false_R
3,3.0,ಕೆಲಸಕ್ಕೆ,I-false_R
4,4.0,ಹೋಗಿ,I-false_R
5,5.0,ಬಂದು,I-false_R
6,6.0,ಹಾ,O
7,7.0,ಹೌದು,O
8,8.0,ಅಂದ್ರೆ,O
9,9.0,ಇರಬೇಕ,O
10,10.0,ಆಗುತ್ತೆ,O


In [ ]:
#extracting features from the train set
Xtrain, ytrain = format_data(train)

In [ ]:
len(ytrain), len(Xtrain)

(6882, 6882)

In [ ]:
# tp process dev set
dev = process_blanklines("/content/drive/MyDrive/disfluency/kannada_dev.tsv")

In [ ]:
dev = dev[:len(dev)-1]

In [ ]:
dev.head(12)

,ID,WORD,TAG
0,1.0,ನಮಸ್ತೆ,O
1,2.0,sir,O
2,3.0,ಮಿಥುನ್,O
3,4.0,sir,O
4,5.0,.,O
5,NaN,NaN,NaN
6,1.0,ನಮಸ್ತೆ,O
7,2.0,"*,",O
8,3.0,ಹಾ,O
9,4.0,ಮತ್ತೇ,O


In [ ]:
#extracting features from the dev set
Xdev, ydev = format_data(dev)

In [ ]:
len(Xdev), len(ydev)

(764, 764)

In [ ]:
try:
    crf.fit(Xtrain, ytrain)
except AttributeError:
    pass

In [ ]:
print('labels : ', list(crf.classes_))

labels :  ['B-false_R', 'I-false_R', 'O', 'B-repeat_R', 'B-Alteration', 'B-edit_R', 'I-repeat_R', 'I-Alteration', 'B-repair_R', 'I-repair_R', 'I-edit_R', 'B-pet_R', 'B-filler_R', 'I-filler_R', 'I-pet_R', '']


In [ ]:
ypred_dev = crf.predict(Xdev)
print('F1 score on the test set = {}\n'.format(metrics.flat_f1_score(ydev, ypred_dev, average='weighted')))
print('Accuracy on the test set = {}\n'.format(metrics.flat_accuracy_score(ydev, ypred_dev)))

F1 score on the test set = 0.8268843522055186

Accuracy on the test set = 0.8584969532836831



In [ ]:
#post-processing to write the results to .tsv file
dev['WORD'].replace('*,', '.', inplace=True)
dev = dev.drop(['ID'], axis=1)
dev = dev[['WORD', 'TAG']].fillna('')
dev = dev.assign(P_tags = flatten(ypred_dev))

In [ ]:
dev.head()

,WORD,TAG,P_tags
0,ನಮಸ್ತೆ,O,O
1,sir,O,O
2,ಮಿಥುನ್,O,O
3,sir,O,O
4,.,O,O


In [ ]:
# # to create .tsv file
# dev.to_csv('/content/drive/MyDrive/Disfluency/Test-Blind/Kannada/MUCS_1.tsv', sep='\t', index=False, encoding='utf-8', header= None)

In [ ]:
# to check the created .tsv file
result = pd.read_csv('/content/drive/MyDrive/Disfluency/Test-Blind/Kannada/MUCS_1.tsv', sep='\t')  #dtype=object, encoding="utf-8")  #, sep='\t')

In [ ]:
# to process on the test set
test = test_process_blanklines("/content/drive/MyDrive/disfluency/kannada_test_blind.tsv")
Xtest = test_format_data(test)
ypred_test = crf.predict(Xtest)

In [ ]:
len(Xtest),len(ypred_test), len(test), len(flatten(ypred_test))

(894, 894, 10133, 10133)

In [ ]:
#post-processing to write the results to .tsv file
test['WORD'].replace('*,', '.', inplace=True)
test = test.drop(['ID'], axis=1)
test = test[['WORD']].fillna('')
test = test.assign(P_tags = flatten(ypred_test))

In [ ]:
# to create .tsv file
test.to_csv('/content/drive/MyDrive/disfluency/MUCS_kan_1.tsv', sep='\t', index=False, encoding='utf-8', header= None)

-------------------------------------------------------------------------------

In [ ]:
# to check the created .tsv file
result = pd.read_csv('/content/drive/MyDrive/disfluency/MUCS_kan_1.tsv', sep='\t')

In [ ]:
result.head()

,Hai,O
0,ದೀಪಾಲಿ,O
1,.,O
2,NaN,NaN
3,Hai,O
4,ಅಶ್ವಿನಿ,O


In [ ]:
# to replace '*,' by '.' in a  'WORD' column
temp = test.index[test['WORD'] == '*,'].tolist()

# to check the number of rows containing '*,'
print((test.WORD=='*,').sum())

0
